<a href="https://colab.research.google.com/github/abdgazalaa241992-cmyk/lab/blob/main/IUG_Practical_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Given the dataset below, perform the required task for each question and display the output.



In [ ]:
transactions = [
    ['A', 'B'],   # T1
    ['A'],        # T2
    ['B'],        # T3
    ['A', 'B'],   # T4
    []            # T5 (neither A nor B)
]
transactions

[['A', 'B'], ['A'], ['B'], ['A', 'B'], []]

# Q1- Perform Apirioi algorithm to get the frequent itemsets given min support = 3 and min conf 80%.

# Q2- Perform FP-Growth algorithm to get the frequent itemsets given min support = 3 and min conf 80%.

# Q3- Perform ECLAT algorithm to get the frequent itemsets given min support = 3 and min conf 80%.

# Q4- Perform Apirioi algorithm to get the frequent itemsets given min support = 3 and min conf 80%.

# Q5- Returns a dict with:
      - counts
      - support
      - confidence (X->Y, Y->X)
      - lift (X->Y)
      - interest (same as lift via alternative formula)
      - contingency table counts (a,b,c,d)
      - h-confidence